# Langchain

This code does a semantic search inside a file

In [1]:
# @title Search in files
Question = "What is artificial intelligence?" # @param {type:"string"}
number_of_citations = 10 # @param {type:"number"}

# Download a test file

If you already have a txt file, create the user_path folder and put it inside

In [2]:
#@title Download a test file
#@markdown If you already have a txt file, create the user_path folder and put it inside

!pip install requests PyPDF2

from IPython.display import clear_output
clear_output()

import os
import requests
from PyPDF2 import PdfReader

def download_pdf(url, destination):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(destination, 'wb') as file:
            file.write(response.content)
        print("PDF downloaded successfully.")
    except Exception as e:
        raise RuntimeError(f"Error during PDF download: {e}")

def pdf_to_txt(pdf_path, txt_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
        print("PDF to TXT conversion completed successfully.")
    except Exception as e:
        raise RuntimeError(f"Error during PDF to TXT conversion: {e}")

def remove_pdf(pdf_path):
    try:
        if os.path.exists(pdf_path):
            os.remove(pdf_path)
            print("PDF file removed successfully.")
    except Exception as e:
        raise RuntimeError(f"Error removing the PDF file: {e}")

def main():
    url = "https://www.europarl.europa.eu/RegData/etudes/STUD/2020/641547/EPRS_STU(2020)641547_EN.pdf"
    user_path = "user_path"
    os.makedirs(user_path, exist_ok=True)

    pdf_path = os.path.join(user_path, "document.pdf")
    txt_path = os.path.join(user_path, "document.txt")

    try:
        download_pdf(url, pdf_path)
        pdf_to_txt(pdf_path, txt_path)
        remove_pdf(pdf_path)
        print("Operation completed successfully.")
    except Exception as error:
        print(f"Error: {error}")

if __name__ == "__main__":
    main()


PDF downloaded successfully.
PDF to TXT conversion completed successfully.
PDF file removed successfully.
Operation completed successfully.


In [3]:
# @title Install dependencies

# Install pyhon 3.10 that will be used within pipenv
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null
!sudo apt install python3.10 python3.10-distutils psmisc -y > /dev/null
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10 > /dev/null

!pip install langchain==0.0.354
!pip install 'sentence_transformers>=2.2.2,<2.3.0'
!pip install chromadb==0.4.23
!pip install 'instructorembedding>=1.0.1'

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is co

In [4]:
embedding_model = "hkunlp/instructor-large"
device = 'cuda'
db_dir = "db_dir_UserData"

import time
start_time = time.time()

from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def _chunk_sources(sources, chunk=True, chunk_size=512, language=None, db_type=None):
    assert db_type is not None
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0, keep_separator=False, separators=["\n\n", "\n", " ", ""])
    source_chunks = splitter.split_documents(sources)

    # currently in order, but when pull from db won't be, so mark order and document by hash
    [x.metadata.update(dict(chunk_id=chunk_id)) for chunk_id, x in enumerate(source_chunks)]

    # assign chunk_id=-1 for original content
    # this assumes, as is currently true, that splitter makes new documents and list and metadata is deepcopy
    [x.metadata.update(dict(chunk_id=-1)) for chunk_id, x in enumerate(sources)]

    # in some cases sources is generator, so convert to list
    return list(sources) + source_chunks

def split_list(input_list, split_size):
    for i in range(0, len(input_list), split_size):
        yield input_list[i:i + split_size]

import glob
from langchain.document_loaders import TextLoader
docs1 = TextLoader(glob.glob("user_path/*")[0], encoding="utf8", autodetect_encoding=True).load()
sources =  _chunk_sources(docs1, chunk=True, chunk_size=512, db_type="chroma")


embedding = HuggingFaceInstructEmbeddings(model_name=embedding_model,
                              model_kwargs={'device': device},
                              encode_kwargs={'normalize_embeddings': True})

import os
if not os.path.exists(db_dir):
  os.makedirs(db_dir)
  print("Create DB")
  max_batch_size = 41666
  sources_batches = split_list(sources, max_batch_size)
  for sources_batch in sources_batches:
    db = Chroma.from_documents(documents=sources_batch, embedding=embedding, persist_directory=db_dir, collection_name = 'UserData')

    db.persist()

  #clear_embedding(db)
  #save_embed(db, use_openai_embedding, hf_embedding_model)
else:
  print("Open DB")
  chroma_class = Chroma
  import chromadb
  api_kwargs = dict(client=chromadb.PersistentClient(path=db_dir))
  db = chroma_class(persist_directory=db_dir, embedding_function=embedding,
                    collection_name="UserData",
                    **api_kwargs)

docs = db.similarity_search_with_score(Question, k=1000, filter={'chunk_id': {'$gte': 0}})
for doc in docs[:number_of_citations]:
  print(doc[1])
  print(doc[0].page_content)
  print("*******")

print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_h

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

max_seq_length  512
Create DB


0.17794254422187805
Executive summary ............................................................................................................ III 
Glossary  ............................................................................................................................ VI 
1. What is artificial intelligence?  ....................................................................................... 1
*******
0.20304878056049347
AI is an umbrella term  including  a wide range of technologies and applications  that have little more 
in common than their apparent  intelligence , a quality which remains very much open to 
interpretation.  Further, we regularly talk about AI that is already in widespread use alongside AI that 
is under development, and even AI that is specula ted to possibly exist in the future. Consequently, 
the term ' AI' is regularly used to refer to any technique , used in any context – real or imagined – as
*******
0.2060544490814209
9 H. Collins, Artifi